# Phylogenetic Distance Calculation from RhizCG 2013

Notes: Calculate phylogenetic distance using phylogeny created from rbcL and matK in phylogenerator 20160209_phylogeny.tre.  

# Init

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(foreach)
library(lsmeans)
library(Hmisc)
library(lme4)
library(lmerTest)
library(ggplot2)
library(plyr)
library(dplyr)
library(tidyr)
library(pbkrtest)
library(gridExtra)
library(reshape2)
library(picante)
library(ape)
library(phyloseq)
library(doParallel)


In [ ]:
%%R
# Initialize corMLPE 
source('/home/bryan/corMLPE/corMLPE.R')

## Set file locations

In [ ]:
%%R
workDir = '/home/bryan/RhizCG/PhyDist/'
treeFile = '/home/bryan/RhizCG/PhyDist/20160209_phylogeny.tre'
NAMFile = '/home/bryan/RhizCG/PhyDist/NAM_founders_distanceMatrix_v2.txt'

### Read  tables table 

In [ ]:
%%R 

NAMdist = read.table(NAMFile, header = TRUE)
#reorder so that NAMdist is in same order as log2fold change matrix
NAMdist = NAMdist[c(1:6, 8, 7, 9, 10), c(1:6, 8, 7, 9, 10)]

In [ ]:
%%R
NAM_hc = hclust(as.dist(NAMdist), "average")
plot(NAM_hc)

In [ ]:
%%R
#Read rarified phyloseq object
F = file.path('/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/physeq/Full-Sparsity3in3_r')
physeq.Full = readRDS(F)

physeq.NAM = subset_samples(physeq.Full, Treatment == "Nneg" & Type == "Maize" & Plant != "75-062")
physeq.NAM

## NAM lines clustered by sampling date

In [ ]:
%%R
temp = sample_data(physeq.NAM) %>% 
    as(., "data.frame") %>% 
    group_by(Plant) %>% 
    summarize(DAY = mean(DAP)) #%>%
row.names(temp) = temp$Plant
dist(temp, method = "euclidean") %>%
    hclust("average") %>%
     plot

## Calculate w-Unifrac distances

In [ ]:
%%R
registerDoParallel(cores=28)
phy.wunif.dist = distance(physeq.NAM, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)


# Data wrangling 

## Create melted dataframes for plot by plot comparisons

In [ ]:
%%R
gd.m = melt(as.matrix(NAMdist))

#set column names and create comparison column
colnames(gd.m) = c("PlantA", "PlantB", "gene_Dist")
gd.m$PlantComp = paste(gd.m$PlantA, gd.m$PlantB, sep = "-")
gd.m %>% head


In [ ]:
%%R
ud = as.matrix(phy.wunif.dist) 
# remove lower triangle to remove duplicate values
ud[ lower.tri( ud, diag = TRUE ) ]  <- NA
head(ud)
ud.m = melt(ud, na.rm= TRUE)
colnames(ud.m) = c("PlotA", "PlotB", "w_Unif_Dist")
head(ud.m) %>% print
dim(ud.m) %>% print

## Create functional distance variable

In [ ]:
%%R
#add growing degree days
DAP = c(36, 53, 57, 61, 72, 79, 84, 88)
GDD = c(795.1, 1089.4, 1136.8, 1199.5, 1394.3, 1512.4, 1573.8, 1625.6)
M_GDD = cbind(DAP, GDD) %>% as.data.frame
M_GDD

Plant = c("ERATE", "HELAN", "B73", "FAGES", "AMAPO", "SORSUD", "SORBI", 
          "ELCOR", "ECHCG", "ABUTH", "SETFA", "GLYMA", "MS71", "Mo18", "Il14H", "Tx303", "Oh7B",
         "B97", "Hp301", "75-062", "Ki3", "H99")
SeedSize_mg = c(.0035, 90, 333, 38, .49, 10.7, 32.39,
               3.2, 1.58, 9.6, 1.8, 174, .264, .224, 0.219, 0.279, 0.207,
               0.240, 0.0869, 0.222, 0.228, 0.231)
SS = cbind(Plant, SeedSize_mg) %>% as.data.frame
head(SS) %>% print
SS$SeedSize_mg = SS$SeedSize_mg %>% as.character %>% as.numeric
(SS)

In [ ]:
%%R
# PCA of maize
sd = as(sample_data(physeq.NAM), "data.frame")
sd$Plant %>% levels %>% print
sd = left_join(sd, SS, by = "Plant")
sd = left_join(sd, M_GDD, by = "DAP")

 fit = prcomp(~SeedSize_mg + M_Resid_logN + M_Resid_logCN + M_Resid_logB, data = sd, 
                 na.action = na.exclude, center = TRUE, scale = TRUE)

summary(fit) %>% print
fit$rotation %>% print
biplot(fit)

In [ ]:
%%R -w 600 -h 500
S = predict(fit)[,1:3]
E_Fit = cbind(sd, S)
row.names(E_Fit) = E_Fit$X.sample
sample_data(physeq.NAM) = E_Fit


p = ggplot(E_Fit, aes(PC1, PC3, colour = Plant, 
                       size = A_Resid_logN), pch = 21)
p = p + geom_point()
#p = p + scale_colour_brewer(palette="Paired")
p = p + ggtitle("PCA of Growth Residuals")
p

In [ ]:
%%R
p = ggplot(E_Fit, aes(PC1, PC2, colour = Plant, 
                       size = A_Resid_logN), pch = 21)
p = p + geom_point()
p = p + scale_colour_brewer(palette="Paired")
p = p + ggtitle("PCA of Growth Residuals")
p

In [ ]:
%%R
#measure distances on PC1
sd = sample_data(physeq.NAM)
PC1dist = dist(sd[, "PC1"], method = "euclidian")
    #melt matrix
    PC1.m = melt(as.matrix(PC1dist))

    #set column names and create comparison column
     colnames(PC1.m) = c("PlotA", "PlotB", "PC1Dist")
    PC1.m$PlotComp = paste(PC1.m$PlotA, PC1.m$PlotB, sep = "-")
    PC1.m %>% head

#measure distances on PC2
PC2dist = dist(sd[,"PC2"], method = "euclidian")
    #melt matrix
    PC2.m = melt(as.matrix(PC2dist))

    #set column names and create comparison column
     colnames(PC2.m) = c("PlotA", "PlotB", "PC2Dist")
    PC2.m$PlotComp = paste(PC2.m$PlotA, PC2.m$PlotB, sep = "-")
    PC2.m %>% head

#measure distances on PC3
PC3dist = dist(sd[,"PC3"], method = "euclidian")
    #melt matrix
    PC3.m = melt(as.matrix(PC3dist))

    #set column names and create comparison column
     colnames(PC3.m) = c("PlotA", "PlotB", "PC3Dist")
    PC3.m$PlotComp = paste(PC3.m$PlotA, PC3.m$PlotB, sep = "-")
    PC3.m %>% head %>% print

#measure distances on residCN
CNdist = dist(sd[,"M_Resid_logCN"], method = "euclidian")
    #melt matrix
    CN.m = melt(as.matrix(CNdist))

    #set column names and create comparison column
     colnames(CN.m) = c("PlotA", "PlotB", "CNdist")
    CN.m$PlotComp = paste(CN.m$PlotA, CN.m$PlotB, sep = "-")
    CN.m %>% head %>% print

#measure distances on residBiomass
Ndist = dist(sd[,"M_Resid_logN"], method = "euclidian")
    #melt matrix
    N.m = melt(as.matrix(Ndist))

    #set column names and create comparison column
     colnames(N.m) = c("PlotA", "PlotB", "Ndist")
    N.m$PlotComp = paste(N.m$PlotA, N.m$PlotB, sep = "-")
    N.m %>% head %>% print

#measure distances on Seed Size
SSdist = dist(log(sd[,"SeedSize_mg"]), method = "euclidian")
    #melt matrix
    SS.m = melt(as.matrix(SSdist))

    #set column names and create comparison column
     colnames(SS.m) = c("PlotA", "PlotB", "SSdist")
    SS.m$PlotComp = paste(SS.m$PlotA, SS.m$PlotB, sep = "-")
    SS.m %>% head %>% print

Fundist = dist(sd[,c("PC1", "PC2", "PC3")], method = "euclidian")
    Fun.m = melt(as.matrix(Fundist))

    #set column names and create comparison column
     colnames(Fun.m) = c("PlotA", "PlotB", "FunDist")
    Fun.m$PlotComp = paste(Fun.m$PlotA, PC3.m$PlotB, sep = "-")
    Fun.m %>% head %>% print
PCA.m = left_join(PC1.m, PC2.m[,c(3,4)], by = "PlotComp")
PCA.m = left_join(PCA.m, PC3.m[,c(3,4)], by = "PlotComp")
PCA.m = left_join(PCA.m, Fun.m[,c(3,4)], by = "PlotComp")
PCA.m = left_join(PCA.m, CN.m[,c(3,4)], by = "PlotComp")
PCA.m = left_join(PCA.m, N.m[,c(3,4)], by = "PlotComp")
PCA.m = left_join(PCA.m, SS.m[,c(3,4)], by = "PlotComp")
head(PCA.m)

## Join distance tables

In [ ]:
%%R
#Join with plant names
ud.m = left_join(ud.m, sd[, c(1,9)], by = c("PlotA" = "X.sample"))
head(ud.m) %>% print
colnames(ud.m)[4] = "PlantA"

ud.m = left_join(ud.m, sd[, c(1,9)], by = c("PlotB" = "X.sample"))
head(ud.m) %>% print
colnames(ud.m)[5] = "PlantB"
ud.m$PlantComp = paste(ud.m$PlantA, ud.m$PlantB, sep = "-")

#Create plot comparison for joining with functional distance matrix
ud.m$PlotComp = paste(ud.m$PlotA, ud.m$PlotB, sep = "-")
head(ud.m) %>% print

## Join with plant phylogenetic distance and functional distance

In [ ]:
%%R
Dist = left_join(ud.m, gd.m[,c(3,4)], by = "PlantComp")
tail(Dist) %>% print

#Join with 
# Join DAP information and library information
Dist = left_join(Dist, sd[,c(1,2,13)], by = c("PlotA" = "X.sample"))
colnames(Dist)[9:10] = c("LibraryA","DAY_A")
tail(Dist) %>% print

Dist = left_join(Dist, sd[,c(1,2,13)], by = c("PlotB" = "X.sample"))
colnames(Dist)[11:12] = c("LibraryB","DAY_B")
tail(Dist) %>% print

Dist = left_join(Dist, PCA.m[,3:10], by = "PlotComp")

# set DAY_A and DAY_B as factor
Dist$DAY_A = as.factor(Dist$DAY_A)
Dist$DAY_B = as.factor(Dist$DAY_B)
Dist$TimeDist = abs(as.numeric(as.character(Dist$DAY_A)) - as.numeric(as.character(Dist$DAY_B)))

#create factor for cross library comps and same day comps
Dist$LibraryComp = ifelse(Dist$LibraryA == Dist$LibraryB, 0, 1) %>% as.factor
Dist$SameDay = ifelse(Dist$DAY_A == Dist$DAY_B, 0, 1) %>% as.factor

head(Dist)

# Build corMLPE models

## corMLPE models without random effects

In [ ]:
%%R
str(Dist)

In [ ]:
%%R
Data = filter(Dist, gene_Dist != 0) 
dim(Dist) %>% print
dim(Data)

In [ ]:
%%R
#Base models
fit1 =  gls(w_Unif_Dist ~ 1, correlation = corMLPE(form= ~ PlotA + PlotB), data = Data)

In [ ]:
%%R
summary(fit1)

In [ ]:
%%R
#Base models
fit2 =  gls(w_Unif_Dist ~ gene_Dist, correlation = corMLPE(form= ~ PlotA + PlotB), data = Data)


In [ ]:
%%R
summary(fit2)

In [ ]:
%%R
#Controlling for time and batch effects, using same correlation structure
fit3 =  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp, 
             correlation = corMLPE(value = 0.2, form= ~ PlotA + PlotB, Z = Z), data = Data)
summary(fit3)


In [ ]:
%%R
#Hypothesis testing functional distance
fit3 =  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(value = 0.2, form= ~ PlotA + PlotB, Z = Z), data = Data)
summary(fit3)

In [ ]:
%%R
#H2: phylogenetic dist
fit4 =  gls(w_Unif_Dist ~  gene_Dist, 
            correlation = corMLPE(form= ~ PlotA + PlotB), data = Data)
summary(fit4)

In [ ]:
%%R
fit5 =  gls(w_Unif_Dist ~   poly(gene_Dist,2), 
             correlation = corMLPE(value = 0.2, form= ~ PlotA + PlotB, Z = Z), data = Data)
summary(fit5)

In [ ]:
%%R
#H3: combined phylogenetic distance and functional
fit6=  gls(w_Unif_Dist ~ TimeDist + gene_Dist + SameDay + LibraryComp + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(value = 0.2, form= ~ PlotA + PlotB, Z = Z), data = Data)
summary(fit6)

In [ ]:
%%R
anova(fit1, fit2, fit3, fit4, fit5, fit6)

Notes: Genetic distance^2 is the best model on NAM lines

In [ ]:
%%R
plot(fit5)

In [ ]:
%%R
test <- function(x) {.18*x + 0.18}


                
p = ggplot(Dist, aes(y = w_Unif_Dist, x = gene_Dist)) 
p = p + geom_point() #aes(size = phy_Dist), shape = 1)
#p = p + stat_smooth(method = "lm", formula = y~.29x -.236x^2 + .204, se = FALSE)
p = p + theme_bw(base_size = 16)
p = p + ggtitle("NAM line genetic distance vs \n microbial community dissimilarity ")
#p = p + stat_smooth(method = lm, se = FALSE)
p = p + stat_function(fun = test, xlim = c(.12, .16))
p

### Not used, but if date/combo is random effect:

In [ ]:
%%R
Dist$DateCombo = ifelse(as.numeric(as.character(Dist$DAY_A)) < as.numeric(as.character(Dist$DAY_B)),
                                                                    paste(Dist$DAY_A, Dist$DAY_B, sep = "-"), 
                                                                   paste(Dist$DAY_B, Dist$DAY_A, sep = "-"))
Dist$DateCombo %>% unique


In [ ]:
%%R

p = ggplot(Dist, aes(x = gene_Dist, y = w_Unif_Dist))
           p = p + geom_point()
        p = p + stat_smooth(method = "lm", se = FALSE)
           p = p + facet_wrap(~DateCombo)
           p

In [ ]:
# GLS model on early subset of plants

In [ ]:
%%R
Dist$DAY_A %>% unique

In [ ]:
%%R
dim(Dist) %>% print
SubDist = filter(Dist, DAY_A %in% c( "53", "57", "61") & DAY_B %in% c( "53", "57", "61"))


dim(SubDist) %>% print
SubDist$PlantA %>% unique

In [ ]:
%%R
#null model
fit1=  gls(w_Unif_Dist ~ 1, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)


In [ ]:
%%R
fit2=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit2)

In [ ]:
%%R
fit3=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + phy_Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit3)

In [ ]:
%%R
fit4=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + poly(phy_Dist,2), 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit4)

In [ ]:
%%R
#H3: combined phylogenetic distance and functional
fit5=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit5)

In [ ]:
%%R
fit6=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + phy_Dist + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit6)

# Subset of late season plants

In [ ]:
%%R
dim(Dist) %>% print
SubDist = filter(Dist, PlantA %in% c("MS71",   "ERATE", 
 "ELCOR",  "SORSUD",    "SORBI", "HELAN",    "Mo18") & PlantB %in% c("MS71", "ERATE", 
 "ELCOR",  "SORSUD",    "SORBI",  "HELAN",  "Mo18"))
dim(SubDist) %>% print
SubDist$PlantA %>% unique

In [ ]:
%%R
#null model
fit1=  gls(w_Unif_Dist ~ 1, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)

In [ ]:
%%R
fit2=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit2)

In [ ]:
%%R
fit3=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + phy_Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit3)

In [ ]:
%%R
fit4=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + poly(phy_Dist,2), 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit4)

In [ ]:
%%R
#H3: combined phylogenetic distance and functional
fit5=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit5)

In [ ]:
%%R
fit6=  gls(w_Unif_Dist ~ TimeDist + SameDay + LibraryComp + poly(phy_Dist,2) + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = SubDist)
summary(fit6)

In [ ]:
%%R
anova(fit1, fit2, fit3, fit4, fit5, fit6)

# Maize distance analysis

In [ ]:
%%R
NAM_hc = hclust(as.dist(NAMdist), "average")
plot(NAM_hc)

In [ ]:
%%R
physeq.Maize = subset_samples(physeq.Full, Type == "Maize" & Plant != "75-062")
as(sample_data(physeq.Maize), "data.frame") %>%

group_by(Plant) %>%
   summarise(DATE = first(DAP)) %>% 
    arrange(DATE) %>%
   print()

In [ ]:
%%R
Dist %>% dim %>% print
#Use melt to join genetic distance matrix with unifrac distance
#head(ud.m) ##Use formated unifrac distance from above
#melt 
NAM.m = melt(as.matrix(NAMdist))

#set column names and create comparison column
colnames(NAM.m) = c("PlantA", "PlantB", "gene_Dist")

NAM.m$PlantComp = paste(NAM.m$PlantA, NAM.m$PlantB, sep = "-")
NAM.m %>% head

#Join with unifrac DistanceMatrix
NAMDist = left_join(Dist, NAM.m[,c(3,4)], by = "PlantComp") %>% filter(!is.na(gene_Dist))
NAMDist %>% dim %>% print
NAMDist %>% head

In [ ]:
%%R
fit1=  gls(w_Unif_Dist ~ 1, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = NAMDist)
summary(fit1)

In [ ]:
%%R
fit2=  gls(w_Unif_Dist ~  SameDay + LibraryComp, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = NAMDist)
summary(fit2)

In [ ]:
%%R
fit3=  gls(w_Unif_Dist ~ SameDay + LibraryComp + gene_Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = NAMDist)
summary(fit3)

In [ ]:
%%R
fit4=  gls(w_Unif_Dist ~  SameDay + LibraryComp + poly(gene_Dist,2), 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = NAMDist)
summary(fit4)

In [ ]:
%%R
fit5=  gls(w_Unif_Dist ~ SameDay + LibraryComp + PC1Dist + PC2Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = NAMDist)
summary(fit5)

In [ ]:
%%R
fit6=  gls(w_Unif_Dist ~ SameDay + LibraryComp + poly(gene_Dist,2) + PC1Dist + PC2Dist + PC3Dist, 
             correlation = corMLPE(form= ~ PlotA + PlotB), data = NAMDist)
summary(fit6)

# Time vs MCC mantel test

In [ ]:
%%R
sample_data(physeq.Nneg)$DAP

In [ ]:
%%R
sd = sample_data(physeq.Nneg)

# Calculate w-unifrac distances
registerDoParallel(cores=28)
date.wunif.dist = distance(physeq.Nneg, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

#Calculate functional distances
Fun_dist = dist(sd[,c("DAP")], method = "euclidian")


M1 = mantel(date.wunif.dist, Fun_dist, method="pearson", permutations=999)
M1

In [ ]:
%%R
mcc.correlog <- mantel.correlog(date.wunif.dist, Fun_dist, nperm=99)
summary(mcc.correlog) %>% print
mcc.correlog  %>% print
plot(mcc.correlog) 

# Ignore below this point

# Compare plant phylogenetic distance to l2f

In [ ]:
%%R
# log2foldFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/DeSeq2/bare-plant.csv'
# l2f = read.csv(log2foldFile, header = TRUE)
# colnames(l2f)

### Spread by plant to make matrix of log2fold change in each OTU by plant combo

In [ ]:
%%R
# df = l2f[,c(1,11,13)]
# l2f_m = spread(df, Plant, log2FoldChange, fill = 0, drop = TRUE)
# rownames(l2f_m) = l2f_m$OTU
# l2f_m = l2f_m[,-1]
# l2f_m %>% head

In [ ]:
%%R
# dim(l2f_m)

### recode all negative log2fold changes with 0

In [ ]:
%%R
# l2f_me = l2f_m
# l2f_me[l2f_me < 0] = 0 
# head(l2f_me)

## Calculate pairwise weighted Unifrac distances between samples

need to figure out how to run unifrac measure outside of phyloseq

In [ ]:
%%R
# physeq.Full

In [ ]:
%%R
# #provide fast tree for unifrac calculation
# #treeFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/fasttree/otusn.tree'
# #Tree = read.tree(treeFile)
# Tree = phy_tree(physeq.Full)
# str(Tree)


In [ ]:
%%R
# l2f_wunif = picante::unifrac(as.matrix(t(l2f_me)), Tree)

In [ ]:
%%R
# l2f_bray = vegdist(as.matrix(t(l2f_me)), method = "bray")

In [ ]:
%%R
# hc_unif = hclust(l2f_wunif, "average")
# plot(hc_unif) %>% print

# hc_bray = hclust(l2f_bray, "average")
# plot(hc_bray, main = "Cluster Dendrogram, \n Bray-Curtis Distance")

# Mantel test of phylogenetic distance and l2f unifrac distance

### Maize

In [ ]:
%%R
# NAMdist %>% print

# #subset column and rows of log2fold change so that only NAM lines are included
# NLU = as.matrix(l2f_wunif)
# NLU = NLU[c(4,5,11,13:18,22),c(4,5,11,13:18,22)]
# NLU

# NLB = as.matrix(l2f_bray)
# NLB = NLB[c(4,5,11,13:18,22),c(4,5,11,13:18,22)]
# NLB

In [ ]:
%%R
# M1 = mantel(as.dist(NAMdist), as.dist(NLU), method="pearson", permutations=999)
# print(M1)

In [ ]:
%%R
# M1 = mantel(as.dist(NAMdist), as.dist(NLB), method="pearson", permutations=999)
# print(M1)

### Species

In [ ]:
%%R

# #Organize log2fold change distance matrix so that only plants with cholorplast distances are included 
# #and column and row order is congruent with phylogenetic distance matrix
# l2f_wunif.t = as.matrix(l2f_wunif) %>% as.data.frame()
#     temp1 = l2f_wunif.t[c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR"), c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR")]

#     SLU = as.matrix(temp1)

#     SLU = as.dist(SLU)
#     SLU %>% print

# #Remove ginkgo so that matrices are congruent
# p.temp = as.matrix(phydist) %>% as.data.frame()
#     temp2 = p.temp[-12, -12] #remove ginkgo

#     temp2 = as.matrix(temp2)
#     phydist = as.dist(temp2)

#     phydist %>% print

# l2f_bray.t = as.matrix(l2f_bray) %>% as.data.frame()
#     temp2 = l2f_bray.t[c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR"), c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR")]

#     SLB = as.matrix(temp2)
#     SLB = as.dist(SLB)
#     SLB %>% print


In [ ]:
%%R
# m1 = mantel(phydist, SLU, method="pearson", permutations=999)
# print(m1)

In [ ]:
%%R
# m1 = mantel(phydist, SLB, method="pearson", permutations=999)
# print(m1)